<a href="https://colab.research.google.com/github/monesh1981/GEE_Projectmkb/blob/main/NDVI_CG_MODIS_VIZ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing the Required Libraries**

ee: The Earth Engine (EE) API, which allows you to access and process satellite imagery and geospatial data.

geemap: A Python package that provides an easy-to-use interface for visualizing and analyzing Earth Engine data with interactive maps (built on folium and ipyleaflet).

**Authenticating with Earth Engine**

This line authenticates Google Earth Engine account.
The first time we run it, it will ask for authentication via my Google account.
Once authenticated, it allows Python to communicate with GEE.

**Initializing the Earth Engine**

Initializes the Earth Engine client.
The project='ee-moneshbathre' argument specifies the GEE Cloud Project in which computations will be performed.

**Creating an Interactive Map**

Creates an interactive map widget using geemap.
We can add satellite imagery, vector layers, and other geospatial datasets to this map.



In [57]:
import ee
import geemap
ee.Authenticate()
ee.Initialize(project = 'ee-moneshbathre')
map = geemap.Map()


** Loading the Feature Collection**

 ee.FeatureCollection(...): Loads a FeatureCollection from your GEE assets.

"projects/ee-moneshbathre/assets/chattisgarh_bdy": This is the path to uploaded asset, which contains the boundary of Chhattisgarh, India.

**Adding the Layer to the Map**

map.addLayer(...) adds the FeatureCollection (cg_bdy) to the interactive geemap map.

{}: Empty dictionary means default styling (no specific color or symbology).
"Chattisgarh_boundary": The name of the layer in the map legend.

**Centering the Map on the Boundary**

Loads the Chhattisgarh boundary from GEE assets.
Displays it on the interactive map.
Centers the map on Chhattisgarh with an appropriate zoom level.




In [58]:
cg_bdy = ee.FeatureCollection("projects/ee-moneshbathre/assets/chattisgarh_bdy")
map.addLayer(cg_bdy,{},"Chattisgarh_boundary")
map.centerObject(cg_bdy,10)

**Loading the MODIS NDVI Data**

ee.ImageCollection("MODIS/061/MOD13A2"): Loads the MODIS Terra Vegetation Indices (MOD13A2) dataset.

select("NDVI"): Extracts only the NDVI band from the dataset.

**Adding NDVI to the Map**

This adds the NDVI data as a layer to the interactive map.
{}: No visualization parameters are defined, so it may not appear correctly.
"ndvi": Layer name.






In [59]:
ndvi = ee.ImageCollection("MODIS/061/MOD13A2").select("NDVI")
map.addLayer(ndvi,{},"ndvi")

In [60]:
region = ee.Geometry.Polygon(
    [[[80.1945, 17.7844],
    [80.1945, 24.1039],
    [84.4094, 24.1039],
    [84.4094, 17.7844],
    [80.1945, 17.7844]]],
    None,
    False)

**adds a new property called "doy" (Day of Year) to each image in the MODIS NDVI collection**

ndvi is an ee.ImageCollection, meaning it contains multiple images over time.
.map(lambda img: ...) applies the function to each image in the collection.

 **Extracting the Day of Year (DOY)**

 img.get('system:time_start'): Retrieves the acquisition timestamp of the image.
ee.Date(...): Converts this timestamp into a date object.
.getRelative('day', 'year'): Extracts the day of the year (DOY) from the date

**Storing the DOY as a Property**

.set('doy', value): Assigns the computed DOY as a new property in each image.
Now, each image in ndvi_india has an extra metadata field called 'doy'.





In [61]:
ndvi_india = ndvi.map(lambda img: img.set('doy',
ee.Date(img.get('system:time_start')).getRelative('day', 'year')))

1. ndvi.map(...) - Applying a Function to Each Image
ndvi is an ImageCollection containing multiple NDVI images over time.
.map(lambda img: ...) applies the function to each image in the collection.
2. Extracting the "Day of Year" (DOY)

img.get('system:time_start') → Retrieves the acquisition date of each image.
ee.Date(...) → Converts the timestamp into an Earth Engine Date object.
.getRelative('day', 'year') → Extracts the day of the year (DOY) (ranging from 1 to 365/366

.set('doy', value) → Assigns the computed DOY as a new property in each image.
Now, each image in ndvi_india has an extra field 'doy'.

In [62]:
distinctDOY = ndvi_india.filterDate('2000-02-18', '2001-02-18'); #filtering the MODIS NDVI ImageCollection (ndvi_india) to include only images from February 18, 2000, to February 18, 2001.

In [63]:
#joins two ImageCollections (distinctDOY and ndvi_india) based on the "doy" (Day of Year) property. This is useful for comparing NDVI images from different years but for the same day of the year.

filter = ee.Filter.equals(leftField = 'doy', rightField = 'doy')
join = ee.Join.saveAll('doy_matches')
joincol = ee.ImageCollection(join.apply(distinctDOY,ndvi_india,filter))

**Mapping Over the Joined Collection**

joincol contains images where each has a list of matched NDVI images ('doy_matches') from different years.

**Extracting NDVI Images for Each DOY**

get('doy_matches'): Retrieves the list of NDVI images (matched by DOY).
ee.ImageCollection.fromImages(...): Converts this list into an ImageCollection.

**Computing the Mean NDVI for Each DOY**

reduce(ee.Reducer.mean()): Computes the mean NDVI across all years for a given DOY.

**Storing the Mean NDVI as an ImageCollectio**n

The output comp is now an ImageCollection where:
Each image represents the average NDVI for a given DOY across multiple years.




In [64]:
# computes the mean NDVI for each "Day of Year" (DOY) across multiple years #Extracting matched NDVI images for the same DOY across different years.
#Averaging NDVI values for that specific DOY.


comp = joincol.map(lambda img: ee.ImageCollection.fromImages(img.get('doy_matches')).reduce(ee.Reducer.mean()))

min': 0.0, 'max': 9000.0

Sets the value range for NDVI visualization.
NDVI values range from -1 to 1, but MODIS NDVI is scaled by 10,000, so it ranges from 0 to 10,000.


In [65]:
visparams = {
  'min': 0.0,
  'max': 9000.0,'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'],}

**create a visualized NDVI image collection where:**

Each image in comp is converted to an RGB visualization using visualize().
The images are clipped to the boundary of ntl_bdy (likely a feature collection representing a specific area).

In [66]:
rgbVis = comp.map(lambda img: img.visualize(bands=['NDVI_mean'], **visparams).clip(cg_bdy))

** Code for GIF Parameters This dictionary defines the parameters for creating an animated GIF from an ImageCollection (e.g., your rgbVis collection of NDVI images).**

'region': region

region defines the geographical area for the GIF. It could be a geometry or bounding box (likely the boundary of cg_bdy or any region you are analyzing).

'dimensions': 600

This defines the size of the output GIF in pixels.
600 means the GIF will be 600x600 pixels in size.
'crs': 'EPSG:4326'

Specifies the coordinate reference system (CRS) for the GIF.
'EPSG:4326' refers to the WGS84 geographic coordinate system (latitude/longitude).
'framesPerSecond': 10

Defines how many frames per second will be in the GIF.
10 FPS means the GIF will have a smooth animation, with each frame displayed for 0.1 seconds.
'format': 'gif'

Specifies that the output format will be a GIF file.




In [73]:
gifparams ={'region': region,
            'dimensions': 600,
            'crs': 'EPSG:4326',
            'framesPerSecond': 10,
            'format': 'gif'}

rgbVis is ImageCollection of visualized NDVI images.
getVideoThumbURL(gifparams) will generate a URL for the animated GIF using the parameters defined in gifparams.

In [74]:
print(rgbVis.getVideoThumbURL(gifparams))

https://earthengine.googleapis.com/v1/projects/ee-moneshbathre/videoThumbnails/8616c08f9abdd6e48c6b9c2326985ad0-092271754b29e327baed7fee5b815a10:getPixels
